In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pickle







         










In [2]:
training_data = "archive (10)/train"
testing_data="archive (10)/test"

In [3]:
# Init FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

In [4]:
X_train = []
y_train = []
X_test=[]
y_test=[]

def extract_landmarks(image):
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.multi_face_landmarks:
        lm = results.multi_face_landmarks[0]
        return [coord for pt in lm.landmark for coord in (pt.x, pt.y)]
    return None

In [5]:
# Read all images and labels
for emotion_label in os.listdir(training_data):
    folder_path = os.path.join(training_data, emotion_label)
    for file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, file)
        img = cv2.imread(img_path)
        if img is None:
            continue
        features = extract_landmarks(img)
        if features:
            X_train.append(features)
            y_train.append(emotion_label)


for emotion_label in os.listdir(testing_data):
    folder_path = os.path.join(testing_data, emotion_label)
    for file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, file)
        img = cv2.imread(img_path)
        if img is None:
            continue
        features = extract_landmarks(img)
        if features:
            X_test.append(features)
            y_test.append(emotion_label)   

In [6]:
# Convert to arrays
X_train = np.array(X_train)
X_test=np.array(X_test)
y_train=np.array(y_train)

y_test = np.array(y_test)

In [8]:
X_train.shape

(26530, 936)

In [7]:
# Train model
model = SVC(kernel='linear', probability=True)
model.fit(X_train, y_train)

# Evaluate
acc = model.score(X_test, y_test)
print(f"Test Accuracy: {acc * 100:.2f}%")

# Save model
with open("emotion_model.pkl", "wb") as f:
    pickle.dump(model, f)

Test Accuracy: 52.05%
